In [2]:
#bridging the python 2 and python 3 gap
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os # saving files
import numpy as np #matrix math

#visualizing data
import matplotlib.pyplot as plt
import matplotlib.cm as cm

#machine learning
import tensorflow as tf

#gausian mixture model for generating data
from data_gmm import GMM_distribution, sample_GMM, plot_GMM
#analyzing data 
from data_utils import shuffle, iter_data

#progress bar
from tqdm import tqdm

#TF-Slim is a lightweight library for defining, training and evaluating models in TensorFlow. It enables defining complex networks quickly and concisely
slim = tf.contrib.slim

#Classes that represent batches of statistical distributions. 
#Each class is initialized with parameters that define the distributions
ds = tf.contrib.distributions

#Create a new graph which compute the targets from the replaced Tensors.

graph_replace = tf.contrib.graph_editor.graph_replace

ImportError: No module named data_gmm

In [ ]:
#hyperparams
""" parameters """
n_epoch = 1000 #number of epcohs
batch_size  = 64
dataset_size = 512
input_dim = 2 #data and labels
latent_dim = 2 
eps_dim = 2


#discriminator
n_layer_disc = 2
n_hidden_disc = 256

#generator 
n_layer_gen = 2
n_hidden_gen= 256

#inference network (generator #2)
n_layer_inf = 2
n_hidden_inf= 256

In [3]:
#save our results to the DiscoGAN folder
""" Create directory for results """
result_dir = 'results/DiscoGAN/'
directory = result_dir
if not os.path.exists(directory):
    os.makedirs(directory)